# **A – Ask the Data**
## Como podemos identificar com precisão transações fraudulentas em um conjunto de dados altamente desbalanceado, minimizando falsos positivos e maximizando a detecção de fraudes, a fim de proteger clientes e instituições financeiras?

# **G – Get the Data**

No projeto, os dados vieram de um dataset público do Kaggle (https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud/data), específico para detecção de fraude em cartões de crédito. Esse conjunto disponibiliza 284.807 transações, das quais apenas 492 são fraudulentas (aprox. 0,17%).

As variáveis de entrada são principalmente atributos anonimizados (V1, V2, ..., V28) resultantes de transformações PCA para proteger a privacidade, além de conter as colunas Time, Amount e a classe alvo (Class: fraudulenta ou legítima).

In [4]:
import pandas as pd
# Load the dataset
df = pd.read_csv('creditcard.csv')
# Inspect the data
print(df.head())
print(df.info())
print(df.describe())

   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

# **E – Explore the Data**

No projeto, começou-se verificando a qualidade dos dados: foi constatado que não havia valores nulos ou ausentes em nenhuma coluna. Além disso, notou-se que as colunas de atributos eram identificadas apenas como V1, V2, etc., devido à anonimização, o que limita a interpretação direta das features. Em termos de distribuição, identificou-se que a variável Amount (valor da transação) possuía uma escala muito diferente das demais features, exigindo normalização. O projeto aplicou um escalonamento (StandardScaler) nessa coluna de montante para trazê-la à mesma ordem de grandeza das outras variáveis. Também foi decidido remover a coluna Time, pois não agregava valor preditivo aparente ao modelo. Durante a exploração, reforçou-se a observação do desbalanceamento das classes – somente 0,17% das instâncias eram fraudes. Esse insight orientou etapas posteriores, pois ficou claro que técnicas especiais seriam necessárias para lidar com o desequilíbrio na modelagem.

In [5]:
# Check for missing values
print(df.isnull().sum())

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64


In [7]:
from sklearn.preprocessing import StandardScaler
# Scale the 'Amount' feature
scaler = StandardScaler()
df['Amount'] = scaler.fit_transform(df[['Amount']])
# Drop 'Time' feature if it's not useful in your model
df = df.drop(columns=['Time'])


# **M – Model the Data**

Primeiramente, o projeto realizou a separação dos dados em treino e teste (usando 80% para treino e 20% para teste), garantindo que essa divisão fosse estratificada. Em seguida, foram aplicadas técnicas para lidar com o desequilíbrio de classes antes e durante a modelagem, já que treinar um modelo na forma bruta dos dados poderia resultar em um classificador tendencioso para a classe majoritária. O artigo menciona abordagens de oversampling, como o SMOTE (Synthetic Minority Over-sampling Technique), que gera novas amostras sintéticas da classe fraudulenta para reforçar sua representatividade no treino. Também são discutidas estratégias de undersampling (reduzir amostras da classe majoritária) e o uso de ajuste de peso de classe nos algoritmos (por exemplo, definir `class_weight='balanced'` em modelos como Regressão Logística ou Random Forest) para dar maior peso às fraudes durante o aprendizado. Adicionalmente, considerando que fraudes são casos raros e “anomalias”, o projeto mencionou técnicas de detecção de anomalias como autoencoders, os quais podem ser treinados apenas nos dados não fraudulentos e identificar transações fora do padrão esperado como potenciais fraudes.

 Foram experimentados modelos clássicos como Regressão Logística e Árvore de Decisão, métodos baseados em instâncias como K-Vizinhos Mais Próximos (KNN), modelos de ensemble incluindo Random Forest e o algoritmo de boosting XGBoost, além de um Autoencoder configurado para detectar anomalias. Cada modelo foi avaliado com métricas apropriadas para problemas de classificação desbalanceada, especialmente Precision (precisão), Recall (sensibilidade), F1-Score e ROC-AUC, além da acurácia tradicional.

In [8]:
from sklearn.model_selection import train_test_split

# Define X and y
X = df.drop(columns=['Class'])  # Features
y = df['Class']  # Target variable

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [12]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Treinar o modelo
model = LogisticRegression(class_weight='balanced', max_iter=1000)
model.fit(X_train, y_train)

# Prever no conjunto de teste
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]  # probabilidades para ROC-AUC

# Avaliar o desempenho
print("Logistic Regression Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.6f}")
print(f"Precision: {precision_score(y_test, y_pred):.6f}")
print(f"Recall: {recall_score(y_test, y_pred):.6f}")
print(f"F1 Score: {f1_score(y_test, y_pred):.6f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_proba):.6f}")

Logistic Regression Performance:
Accuracy: 0.974562
Precision: 0.058785
Recall: 0.918367
F1 Score: 0.110497
ROC AUC: 0.971447


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd

# Calcular o peso para XGBoost
ratio = (y_train == 0).sum() / (y_train == 1).sum()

# Dicionário com os modelos
modelos = {
    "Logistic Regression": LogisticRegression(class_weight='balanced', max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(class_weight='balanced'),
    "Random Forest": RandomForestClassifier(class_weight='balanced'),
    "XGBoost": XGBClassifier(scale_pos_weight=ratio, use_label_encoder=False, eval_metric='logloss'),
    "K-Nearest Neighbors": KNeighborsClassifier(),
}

# Avaliação
resultados = []

for nome, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    try:
        y_proba = modelo.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, y_proba)
    except:
        roc_auc = None

    resultados.append({
        "Modelo": nome,
        "Accuracy": round(accuracy_score(y_test, y_pred), 6),
        "Precision": round(precision_score(y_test, y_pred), 6),
        "Recall": round(recall_score(y_test, y_pred), 6),
        "F1 Score": round(f1_score(y_test, y_pred), 6),
        "ROC AUC": round(roc_auc, 6) if roc_auc is not None else None
    })

# Mostrar resultados em tabela
df_resultados = pd.DataFrame(resultados)
print(df_resultados)


                Modelo  Accuracy  Precision    Recall  F1 Score   ROC AUC
0  Logistic Regression  0.974562   0.058785  0.918367  0.110497  0.971447
1        Decision Tree  0.999122   0.755319  0.724490  0.739583  0.862043
2        Random Forest  0.999526   0.949367  0.765306  0.847458  0.958164
3              XGBoost  0.999544   0.882979  0.846939  0.864583  0.969123
4  K-Nearest Neighbors  0.999508   0.897727  0.806122  0.849462  0.943758


# **C – Communicate the Data**

Communicate the Data envolve interpretar os resultados obtidos e comunicá-los de forma clara. O projeto exemplificou bem essa etapa ao analisar os desempenhos de cada modelo e traduzir as métricas em implicações de negócio. No artigo, após apresentar os números de precisão, recall, etc., foram feitas explicações em texto comum destacando os trade-offs.

Outra forma de comunicação de insights foi a explicação das métricas de desempenho em termos leigos e seu contexto. O texto dedicou uma seção para definir Acurácia, Precisão, Recall, F1 e ROC-AUC, junto com interpretações relevantes ao caso de uso. Por exemplo, enfatizou-se que, em detecção de fraude, uma alta precisão é importante para não rotular indevidamente muitas transações legítimas como fraudulentas (evitar falsos positivos), enquanto uma alta sensibilidade (recall) é crucial para não deixar fraudes reais passarem despercebidas.